In [30]:
import numpy as np
import pandas as pd
import datetime, time
import json
import requests
import 

In [31]:
last_date = datetime.datetime.utcfromtimestamp(time.time())

# Data Collection of Reddit Scraping via Pushshift API

The data collection process will be done through Pushshift's API to scrape both askphilosophy and psychonauts subreddits. The loop starts from 2 days before todays date. The scrape will go until 15000 posts are collecetted with 2 second rests between each requests. The collected data will be added to a dataframe and saved. This occurs for each subreddit.

In [ ]:
#Credit to Gwen Rathgeber/Ben Mathis! Thank you so much.

subreddits = ['Psychonaut','askphilosophy']
kind = "submission"  # we want text posts

# Establish URL base
BASE_URL = f"https://api.pushshift.io/reddit/search/{kind}" # also known as the "API endpoint"

last_date = datetime.datetime.utcfromtimestamp(time.time())     #utc from timestamp -50_000
posts = {}  #empty dictionary
for subreddit in subreddits:
    posts[subreddit] = []
    day = 2                  #start with the most recent post
    cumulative_posts = 0
    while cumulative_posts < 15000:                           #scrape 15,000 b/c minimum is 10,000 and some will be junk from what you scrape
        stem = f"{BASE_URL}?subreddit={subreddit}&size=100"   #part of query, #will scrape from 100 posts
        URL = f"{stem}&after={day}d"                           #will scrape from after the day we scrape it
        print("Querying from: " + URL)
        try:                                                  #we use try, except b/c scraping from the web, you'll get a lot of errors
            res = requests.get(URL)
            assert res.status_code == 200
            json = res.json()['data']
            df = pd.DataFrame(json)
            posts[subreddit].append(df)
            cumulative_posts += df.shape[0]
            final_date_pulled = datetime.datetime.utcfromtimestamp(df.iloc[-1, df.columns.get_loc('created_utc')])
            increment = (last_date - final_date_pulled).days + 1
            increment = increment if increment > 0 else 1
            day += increment
            last_date = final_date_pulled
            print('successful')
        except:
            print(f'Scrape for {URL}, {day} failed')

        time.sleep(2)                    #this is a delay in between scrapes

print("Query complete!")

psychonaut_frame = pd.concat(posts['Psychonaut'])
philosophy_frame = pd.concat(posts['askphilosophy'])

psychonaut_frame.to_csv('./data/raw_psychonaut_initial_scrape.csv')
philosophy_frame.to_csv('./data/raw_askphilosophy_initial_scrape.csv')